<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Trábajo Práctico Nro. 1 
## Alumno: Ezequiel Alejandro Caamaño
## a1802

In [3]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [5]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [6]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [7]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [8]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [9]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [10]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [11]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [12]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [13]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [14]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [15]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [16]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [17]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [21]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [22]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

## Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# **1**. Resolvemos el punto 1.

Empecemos con los siguientes ids:
idx1 = 1111
idx2 = 2222
idx3 = 3333
idx4 = 4444
idx5 = 2323


In [24]:
# Veamos similaridad de documentos. Tomemos el documento 1111
idx1 = 1111
print(newsgroups_train.data[idx1])

Hi netters,

My friend is seriously thinking of getting the Subaru SVX. There is
a local dealer here in Seattle selling them for $22600, with
Touring package, that's $7400 off from MSRP. He thinks it's a 
very good deal (and I think so too). Since he knows I have access to
the net, he would like to get anyone's opinion about this car, especially
in the area of reliability and maintenanability.
Please send e-mail to me as my friend doesn't have access to the net.

My opinion about this car is, you get a lot for $22600:
auto everything (tranny, climate control, windows, locks, folddow rear seet),
full wheel drive, 2+2, fast (143 top spped), heavy (3580lb);-)

Thanks in advacne!


In [25]:
# midamos la similaridad coseno con todos los documentos de train
cossim1 = cosine_similarity(X_train[idx1], X_train)[0]

In [26]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim1)[::-1]

array([1.        , 0.2463245 , 0.19692644, ..., 0.        , 0.        ,
       0.        ])

In [27]:
# y a qué documentos corresponden
np.argsort(cossim1)[::-1]

array([ 1111,  9491,   120, ...,  6724,  6716, 10829])

In [28]:
# los 5 documentos más similares:
mostsim1 = np.argsort(cossim1)[::-1][1:6]

In [29]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx1]]

'rec.autos'

In [30]:
# y los 5 más similares son de las clases:
for i in mostsim1:
  print(newsgroups_train.target_names[y_train[i]])

rec.autos
soc.religion.christian
rec.autos
soc.religion.christian
rec.autos


In [31]:
# Veamos similaridad de documentos. Tomemos el documento 2222
idx2 = 2222
print(newsgroups_train.data[idx2])


however, the word "pa^ques" in french _is_ the word for easter.  ask
any francophone, whether from quebec or from paris.  besides, haven't
you heard of the phrase "the paschal lamb" (meaning jesus)?

sorry to nitpick on the more trivial part of this thread....


In [32]:
# midamos la similaridad coseno con todos los documentos de train
cossim2 = cosine_similarity(X_train[idx2], X_train)[0]

In [33]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim2)[::-1]

array([1.        , 0.25658245, 0.20773143, ..., 0.        , 0.        ,
       0.        ])

In [34]:
# y a qué documentos corresponden
np.argsort(cossim2)[::-1]

array([2222, 3853, 5986, ..., 8484,  660, 7207])

In [35]:
# los 5 documentos más similares:
mostsim2 = np.argsort(cossim2)[::-1][1:6]

In [36]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx2]]

'soc.religion.christian'

In [37]:
# y los 5 más similares son de las clases:
for i in mostsim2:
  print(newsgroups_train.target_names[y_train[i]])

soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
alt.atheism


In [38]:
# Veamos similaridad de documentos. Tomemos el documento 3333
idx3 = 6666
print(newsgroups_train.data[idx3])

Hello,

	Has anyone built cxterm (X11R5) on a MIPS platform.  If you have,
please send me email as I don't read this group.  I've a bunch of questions...
:-)

				Thanks


In [39]:
# midamos la similaridad coseno con todos los documentos de train
cossim3 = cosine_similarity(X_train[idx3], X_train)[0]

In [40]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim3)[::-1]

array([1.        , 0.24077394, 0.24077394, ..., 0.        , 0.        ,
       0.        ])

In [41]:
# y a qué documentos corresponden
np.argsort(cossim3)[::-1]

array([6666, 8719, 2429, ..., 1241, 6489, 8078])

In [42]:
# los 5 documentos más similares:
mostsim3 = np.argsort(cossim3)[::-1][1:6]

In [43]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx3]]

'comp.windows.x'

In [44]:
# y los 5 más similares son de las clases:
for i in mostsim3:
  print(newsgroups_train.target_names[y_train[i]])

sci.med
comp.graphics
talk.religion.misc
talk.politics.misc
sci.electronics


In [45]:
# Veamos similaridad de documentos. Tomemos el documento 1111
idx4 = 4444
print(newsgroups_train.data[idx4])


It was more than a theoretical concept; it was seriously pursued by
Freeman Dyson et al many years ago. I don't know how well-known this is,
but a high explosive Orion prototype flew (in the atmosphere) in San
Diego back in 1957 or 1958. I was working at General Atomic at the time,
but I didn't learn about the experiment until almost thirty years later,
when 
Ted Taylor visited us and revealed that it had been done. I feel sure
that someone must have film of that experiment, and I'd really like to
see it. Has anyone out there seen it?


In [46]:
# midamos la similaridad coseno con todos los documentos de train
cossim4 = cosine_similarity(X_train[idx4], X_train)[0]

In [47]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim4)[::-1]

array([1.        , 0.24766676, 0.24031326, ..., 0.        , 0.        ,
       0.        ])

In [48]:
# y a qué documentos corresponden
np.argsort(cossim4)[::-1]

array([ 4444,  8483,  9623, ...,  9450,  9452, 10169])

In [49]:
# los 5 documentos más similares:
mostsim4 = np.argsort(cossim4)[::-1][1:6]

In [50]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx4]]

'sci.space'

In [51]:
# y los 5 más similares son de las clases:
for i in mostsim4:
  print(newsgroups_train.target_names[y_train[i]])

sci.space
talk.politics.mideast
talk.politics.mideast
talk.politics.mideast
talk.politics.mideast


In [52]:
# Tomemos el documento 2323
idx5 = 2323
print(newsgroups_train.data[idx5])


The Fujitsu 2322 uses what is known as an 'SMD' interface (Storage Module
Device?).  A lot of older minis used it.  Sun still does on their server
models.  There are several different speeds of SMD, and I think that the 
Fuji drive you have is rated at about 24 Mb/sec (thats megabits).  There
used to be several companies that made couplers for the PC (Interphase in
Texas being one), but I think that the market pretty much has dried up.
Controllers for this type of drive are readily available for VME buses
though.  Rumor has it that there is a SMD to SCSI adapter available, but
I think that it was designed for slower SMD devices.  In other words,
if you have a PC or Mac, that drive is pretty much dogmeat.
Cheers,
Corwin



In [53]:
# midamos la similaridad coseno con todos los documentos de train
cossim5 = cosine_similarity(X_train[idx5], X_train)[0]

In [54]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim5)[::-1]

array([1.        , 0.38013379, 0.22989429, ..., 0.        , 0.        ,
       0.        ])

In [55]:
# y a qué documentos corresponden
np.argsort(cossim5)[::-1]

array([2323, 5908, 7722, ..., 5927, 8033, 6115])

In [56]:
# los 5 documentos más similares:
mostsim5 = np.argsort(cossim5)[::-1][1:6]

In [57]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx5]]

'sci.electronics'

In [58]:
# y los 5 más similares son de las clases:
for i in mostsim5:
  print(newsgroups_train.target_names[y_train[i]])

sci.electronics
comp.sys.ibm.pc.hardware
comp.sys.ibm.pc.hardware
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware


**2**. Resolvemos el punto 2.

Empecemos por importar las librerìas que vamos a necesitar



In [59]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

from sklearn.datasets import fetch_20newsgroups


Cargamos los datos, sets de entrenamiento y prueba



In [60]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train, y_train = newsgroups_train.data, newsgroups_train.target
X_test, y_test = newsgroups_test.data, newsgroups_test.target

print(f"Número de documentos de entrenamiento: {len(X_train)}")
print(f"Número de documentos de prueba: {len(X_test)}")


Número de documentos de entrenamiento: 11314
Número de documentos de prueba: 7532


Vectorizamos

In [61]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"Forma de la matriz TF-IDF de entrenamiento: {X_train_tfidf.shape}")
print(f"Forma de la matriz TF-IDF de prueba: {X_test_tfidf.shape}")

Forma de la matriz TF-IDF de entrenamiento: (11314, 101631)
Forma de la matriz TF-IDF de prueba: (7532, 101631)


Comenzamos con la resolución, buscando optimmizar NB

In [62]:
# Definir los modelos a probar
models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

# Definir diferentes configuraciones para el TfidfVectorizer
# Experimentaremos con ngram_range y min_df/max_df para ver el impacto
vectorizer_params = {
    'Base': {}, # Sin parámetros adicionales
    'Ngrams (1,2)': {'ngram_range': (1, 2)}, # Unigramas y bigramas
    'Filtered (min_df=5, max_df=0.8)': {'min_df': 5, 'max_df': 0.8}, # Filtrar palabras muy raras y muy comunes
    'Ngrams + Filtered': {'ngram_range': (1, 2), 'min_df': 5, 'max_df': 0.8}
}

best_f1_score = -1
best_config_name = ""
best_model_name = ""
best_model_instance = None
best_vectorizer_instance = None

for vec_name, vec_params in vectorizer_params.items():
    print(f"\n--- Probando con TfidfVectorizer: {vec_name} ---")
    current_vectorizer = TfidfVectorizer(**vec_params)

    # Re-vectorizar los datos para cada configuración
    X_train_current_tfidf = current_vectorizer.fit_transform(X_train)
    X_test_current_tfidf = current_vectorizer.transform(X_test)

    for model_name, model_instance in models.items():
        print(f"  Entrenando y evaluando: {model_name}")

        # Entrenar el modelo
        model_instance.fit(X_train_current_tfidf, y_train)

        # Realizar predicciones
        y_pred = model_instance.predict(X_test_current_tfidf)

        # Calcular f1-score macro
        f1 = f1_score(y_test, y_pred, average='macro')

        print(f"    F1-score (macro): {f1:.4f}")
        print("    Reporte de Clasificación:")
        print(classification_report(y_test, y_pred, target_names=newsgroups_test.target_names))

        # Actualizar el mejor modelo si se supera el f1-score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_config_name = vec_name
            best_model_name = model_name
            best_model_instance = model_instance # Guardar la instancia del modelo entrenado
            best_vectorizer_instance = current_vectorizer # Guardar la instancia del vectorizador usado

print(f"\n--- MEJORES RESULTADOS ENCONTRADOS ---")
print(f"Mejor F1-score (macro): {best_f1_score:.4f}")
print(f"Configuración del Vectorizador: {best_config_name}")
print(f"Modelo Naïve Bayes: {best_model_name}")


--- Probando con TfidfVectorizer: Base ---
  Entrenando y evaluando: MultinomialNB
    F1-score (macro): 0.5854
    Reporte de Clasificación:
                          precision    recall  f1-score   support

             alt.atheism       0.81      0.07      0.13       319
           comp.graphics       0.72      0.62      0.67       389
 comp.os.ms-windows.misc       0.70      0.50      0.59       394
comp.sys.ibm.pc.hardware       0.55      0.75      0.64       392
   comp.sys.mac.hardware       0.81      0.61      0.69       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.86      0.69      0.77       390
               rec.autos       0.82      0.68      0.74       396
         rec.motorcycles       0.89      0.63      0.73       398
      rec.sport.baseball       0.95      0.69      0.80       397
        rec.sport.hockey       0.59      0.90      0.71       399
               sci.crypt       0.47      0.80      0.59       39

# Resolvemos el punto 3

**A** Importamos librerías y recalculamos la matriz  con el mejor vectorizador encontrado

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np # Asegurarse de que numpy esté importado

matrix_document_term_for_similarity = best_vectorizer_instance.transform(X_train)
vectorizer_for_similarity = best_vectorizer_instance


Transponemos la matriz documento-término y obtenemos las características del mejor vectorizador

In [65]:
# 1. Transponer la matriz documento-término
matrix_term_document = matrix_document_term_for_similarity.T
print(f"Forma de la matriz término-documento: {matrix_term_document.shape}")

# 2. Obtener los nombres de las características (palabras) del mejor vectorizador
feature_names = vectorizer_for_similarity.get_feature_names_out()
print(f"Cantidad de palabras en el vocabulario: {len(feature_names)}")


Forma de la matriz término-documento: (101631, 11314)
Cantidad de palabras en el vocabulario: 101631


Definimos una funciòn que nos permita encontrar las palabras màs similares

In [66]:
# Función para encontrar las palabras más similares
def find_similar_words(word, matrix, feature_names, top_n=5):
    if word not in feature_names:
        # Manejar palabras que no están en el vocabulario
        print(f"La palabra '{word}' no se encuentra en el vocabulario del vectorizador usado.")
        return []

    # Obtener el índice de la palabra
    word_index = np.where(feature_names == word)[0][0]
    word_vector = matrix[word_index]

    # Calcular la similaridad coseno entre el vector de la palabra y todos los demás
    similarities = cosine_similarity(word_vector.toarray(), matrix.toarray())

    # Obtener los índices de las palabras más similares (excluyendo la palabra misma)
    # Convertir a 1D, ordenar de mayor a menor y tomar los top_n+1 (para excluir la palabra original)
    similar_indices = similarities.flatten().argsort()[-top_n-1:-1][::-1]

    similar_words_with_scores = []
    for idx in similar_indices:
        similar_words_with_scores.append((feature_names[idx], similarities[0, idx]))
    return similar_words_with_scores

Elegimos cinco palabras que podamos utilizar relacionadas con temas diversos.

In [67]:
# 3. Elegir 5 palabras manualmente y estudiar sus 5 más similares
selected_words = [
    'computer',   
    'religion',   
    'game',       
    'windows',    
    'israel'  
]

Con la función desarrollada se buscan las palabras similares a esas cinco elegidas

In [68]:
print("\n--- Palabras seleccionadas y sus 5 más similares ---")
for word in selected_words:
    print(f"\nPalabras más similares a '{word}':")
    similar_results = find_similar_words(word, matrix_term_document, feature_names, top_n=5)
    if similar_results:
        for sim_word, score in similar_results:
            print(f"- {sim_word} (Similaridad: {score:.4f})")
    else:
        print(f"No se encontraron palabras similares para '{word}'.")


--- Palabras seleccionadas y sus 5 más similares ---

Palabras más similares a 'computer':
- decwriter (Similaridad: 0.1563)
- harkens (Similaridad: 0.1522)
- deluged (Similaridad: 0.1522)
- shopper (Similaridad: 0.1443)
- the (Similaridad: 0.1361)

Palabras más similares a 'religion':
- religious (Similaridad: 0.2451)
- religions (Similaridad: 0.2116)
- categorized (Similaridad: 0.2039)
- purpsoe (Similaridad: 0.2008)
- crusades (Similaridad: 0.1987)

Palabras más similares a 'game':
- games (Similaridad: 0.2116)
- espn (Similaridad: 0.1935)
- hockey (Similaridad: 0.1806)
- team (Similaridad: 0.1786)
- scored (Similaridad: 0.1707)

Palabras más similares a 'windows':
- dos (Similaridad: 0.3037)
- ms (Similaridad: 0.2320)
- microsoft (Similaridad: 0.2219)
- nt (Similaridad: 0.2140)
- for (Similaridad: 0.1930)

Palabras más similares a 'israel':
- israeli (Similaridad: 0.4169)
- arab (Similaridad: 0.3734)
- syrian (Similaridad: 0.3691)
- lebanon (Similaridad: 0.3626)
- lebanese (Simila

## Conclusiones

Punto 1: Se puede observar claramente que al probar similaridad de distintos documentos, en algunos textos el modelo inicial no arroja resultados tan precisos, obteniendo, por ejemplo, tres clases iguales a las del documento objetivo. Sin embargo, en la mayoría si se obtuvieron buenos resultados, aunque en algunos casos pudiera parecer que no. Ejemplo de esto es el caso del documento , que pertenece a la clase sci.electronics, y los similares a hardware. Más allá de la aparente diferencia, entendemos que son áreas de conocimiento que se solapan, por lo cual el resultado (al poner foco siempre en el dominio del problema) es correcto. 

Algún documento no obtuvo similaridades, como el caso del documento 4, que estaba en la categoría "sci.space" y encontraron similaridades en documentos de la categorìa "politics". Se podría realizar un análisis del contenido de los documentos encontrados, ya que se piensa la carrera espacial en Estados Unidos como política de estado, con lo que podría haber en el texto referencias a esta, pero escapa un poco del foco del trabajo. 

Punto 2: Se efectuaron entrenamiento con modelos de Naice Banes Multinomial y ComplementNB, obteniendo mejores métricas con este último.

Punto 3: Se buscaron similaridad de palabras de distintas áreaa (computación, juegos, sistemas operativos, religion, etc. y se obtuvieron algunos resultados interesantes. Por ejemplo, en el caso de "religion, lo primero que surge son términos derivados, mientras que en el caso de "windows", solo arroja resultados relacionados al contexto, o como el caso de "israel", donde arroja varios gentilicios, pero de lugares relacionados geográficamente e historicamente con ese país. 